### MLP를 통한 부산항 미세먼지 예측
- 산업데이터 과학 수업 데이터
- PM2만 예측하는거임 (task => 다중분류 : 레이블 3개 => 0,1,2 로 타겟값 분류)
- 내가갖고있는 정형데이터를 pytorch에 어떻게 적용하는지 싶을떄 보면 좋음!

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
path = r'C:\Users\SeoJeongBin\Desktop\Code\Pytorch\Pytorch_exercise\data\iudustrial_science\fine_dust.csv'
df = pd.read_csv(path)
df.head()

,PM,PM2,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,...,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,27.0,25.0,1305.26,15069.82,12815.28,2847.84,2847.84,1230.46,14206.22,12080.88,...,636.48,583.44,6736.08,5728.32,1272.96,1272.96,91,7.2,0.906308,0.6
1,52.0,17.0,1021.02,11788.14,10024.56,2227.68,2227.68,729.30,8420.10,7160.40,...,0.00,437.58,5052.06,4296.24,954.72,954.72,36,14.3,0.438371,1.3
2,22.0,12.0,867.68,10017.76,8519.04,1893.12,1893.12,1301.52,15026.64,12778.56,...,318.24,291.72,3368.04,2864.16,636.48,636.48,38,6.7,0.992546,0.5
3,23.0,13.0,729.30,8420.10,7160.40,1591.20,1591.20,729.30,8420.10,7160.40,...,0.00,145.86,1684.02,1432.08,318.24,318.24,23,9.7,0.241922,3.0
4,90.0,63.0,946.22,10924.54,9290.16,2064.48,2064.48,575.96,6649.72,5654.88,...,0.00,583.44,6736.08,5728.32,1272.96,1272.96,18,5.9,0.484810,1.3


In [3]:
# PM2를 등급별로 새로 레이블 매기기

idx0 = df.loc[df['PM2']<=15,:].index # 이러면 행 번호가 나옴
idx1 = df.loc[(df['PM2']>=15) & (df['PM2']<=35),:].index
idx2 = df.loc[(df['PM2']>=36),:].index

df.iloc[idx0,1] = 0
df.iloc[idx1,1] = 1
df.iloc[idx2,1] = 2

df.head()

,PM,PM2,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,...,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,27.0,1.0,1305.26,15069.82,12815.28,2847.84,2847.84,1230.46,14206.22,12080.88,...,636.48,583.44,6736.08,5728.32,1272.96,1272.96,91,7.2,0.906308,0.6
1,52.0,1.0,1021.02,11788.14,10024.56,2227.68,2227.68,729.30,8420.10,7160.40,...,0.00,437.58,5052.06,4296.24,954.72,954.72,36,14.3,0.438371,1.3
2,22.0,0.0,867.68,10017.76,8519.04,1893.12,1893.12,1301.52,15026.64,12778.56,...,318.24,291.72,3368.04,2864.16,636.48,636.48,38,6.7,0.992546,0.5
3,23.0,0.0,729.30,8420.10,7160.40,1591.20,1591.20,729.30,8420.10,7160.40,...,0.00,145.86,1684.02,1432.08,318.24,318.24,23,9.7,0.241922,3.0
4,90.0,2.0,946.22,10924.54,9290.16,2064.48,2064.48,575.96,6649.72,5654.88,...,0.00,583.44,6736.08,5728.32,1272.96,1272.96,18,5.9,0.484810,1.3


In [4]:
# 데이터 셋 분류
x = df.iloc[:,2:]
y = df.iloc[:, 1]

# x : 정규화 수행
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x)
x = min_max_scaler.transform(x)

from sklearn.model_selection import train_test_split
trainx, validx, trainy, validy = train_test_split(x, y, test_size=0.3, random_state = 15)

# x : torch tensor로 타입 변경
trainx = torch.FloatTensor(trainx)
validx = torch.FloatTensor(validx)

# y : torch tensor로 타입 변경
trainy = torch.FloatTensor(trainy.values).view(-1)
trainy = trainy.long()

validy = torch.FloatTensor(validy.values).view(-1)
validy = validy.long()

In [5]:
class myMLP(nn.Module):
    def __init__(self, input_size) : # 여기 같은경우엔 인자로 input size를 받네
        super(myMLP, self).__init__()
        self.input_size = input_size
        self.layer1 = nn.Sequential(
            nn.Linear(self.input_size, 18),
            torch.nn.BatchNorm1d(18), # 배치정규화 좀 알아보기
            nn.Linear(18, 9),
            torch.nn.BatchNorm1d(9),
            nn.Linear(9, 3), # 3 : 예측해야할 레이블 수
            torch.nn.ReLU()
        )
        self.layer2 = torch.nn.Dropout()
        self.layer3 = torch.nn.Softmax()
        
    def forward(self, x):
        output = self.layer1(x)
        output = self.layer2(output)
        output = self.layer3(output)
        return output

In [6]:
# 모델 선언
# trainx.size() : torch.Size([2399, 24])
input_size = trainx.size()[1] # ★ mlp 모양에서 각 노드가 칼럼이 되는셈이라 칼럼개수가 input size다!!!!!!!!
model = myMLP(input_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [7]:
# 학습 돌리기
epochs = 10000

for e in range(epochs):
    optimizer.zero_grad()
    out = model(trainx)
    L = loss(out,trainy)
    L.backward()
    optimizer.step()
    if (e+1) % 1000 == 0:
        with torch.no_grad():
            model.eval()
            pred = model(validx)
            acc = sum(torch.max(pred,1)[1] == validy).numpy()/len(validy) # 이건 task가 분류임을 주의

        print('[Epoch: {}/{}] [Loss: {:2f}] [valid Accuracy: {:2f}]'.format(e+1, epochs, L.item(), acc*100))

C:\Users\SeoJeongBin\AppData\Local\Temp\ipykernel_5428\1789303409.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.layer3(output)


[Epoch: 1000/10000] [Loss: 1.041819] [valid Accuracy: 51.797862]
[Epoch: 2000/10000] [Loss: 0.969562] [valid Accuracy: 57.628766]
[Epoch: 3000/10000] [Loss: 0.969188] [valid Accuracy: 57.920311]
[Epoch: 4000/10000] [Loss: 0.969029] [valid Accuracy: 58.017493]
[Epoch: 5000/10000] [Loss: 0.968922] [valid Accuracy: 57.725948]
[Epoch: 6000/10000] [Loss: 0.968698] [valid Accuracy: 57.920311]
[Epoch: 7000/10000] [Loss: 0.968776] [valid Accuracy: 57.628766]
[Epoch: 8000/10000] [Loss: 0.968532] [valid Accuracy: 57.725948]
[Epoch: 9000/10000] [Loss: 0.968331] [valid Accuracy: 57.531584]
[Epoch: 10000/10000] [Loss: 0.968439] [valid Accuracy: 57.823129]


### test 데이터에 적용해보기

In [8]:
path = r'C:\Users\SeoJeongBin\Desktop\Code\Pytorch\Pytorch_exercise\data\iudustrial_science\fine_dust_without_y.csv'
df_test = pd.read_csv(path)
df_test.head()

,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,...,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,583.44,6736.08,5728.32,1272.96,1272.96,583.44,6736.08,5728.32,1272.96,1272.96,...,954.72,437.58,5052.06,4296.24,954.72,954.72,68,23.7,0.882948,0.7
1,875.16,10104.12,8592.48,1909.44,1909.44,1151.92,13299.44,11309.76,2513.28,2513.28,...,954.72,583.44,6736.08,5728.32,1272.96,1272.96,34,3.5,0.190809,3.5
2,729.30,8420.10,7160.40,1591.20,1591.20,1092.08,12608.56,10722.24,2382.72,2382.72,...,0.00,437.58,5052.06,4296.24,954.72,954.72,58,11.9,0.754710,0.6
3,875.16,10104.12,8592.48,1909.44,1909.44,1600.72,18481.04,15716.16,3492.48,3492.48,...,954.72,291.72,3368.04,2864.16,636.48,636.48,25,7.0,0.275637,1.9
4,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,...,636.48,1301.52,15026.64,12778.56,2839.68,2839.68,66,13.7,-0.173648,0.4


In [9]:
test = df_test 
# x : 정규화 수행
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(test)
test = min_max_scaler.transform(test)

# x : torch tensor로 타입 변경
test = torch.FloatTensor(test)


In [10]:
predict = model(test)
print(len(predict))
predict

100


C:\Users\SeoJeongBin\AppData\Local\Temp\ipykernel_5428\1789303409.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.layer3(output)


tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.6159e-24, 7.9719e-01, 2.0281e-01],
        [1.9890e-16, 9.3983e-01, 6.0168e-02],
        [3.8625e-31, 7.1422e-01, 2.8578e-01],
        [7.5253e-24, 2.3910e-01, 7.6090e-01],
        [4.9442e-23, 8.6994e-01, 1.3006e-01],
        [1.2118e-18, 9.4340e-01, 5.6598e-02],
        [2.9402e-23, 7.9291e-01, 2.0709e-01],
        [7.2385e-37, 5.5710e-01, 4.4290e-01],
        [7.9807e-31, 5.9586e-01, 4.0414e-01],
        [1.0000e+00, 0.0000e+00, 0.0000e+00],
        [7.2630e-37, 5.7520e-01, 4.2480e-01],
        [8.8912e-39, 7.9767e-01, 2.0233e-01],
        [2.5509e-34, 6.8258e-01, 3.1742e-01],
        [3.2614e-18, 7.1989e-01, 2.8011e-01],
        [2.2016e-28, 8.2890e-01, 1.7110e-01],
        [1.3093e-23, 6.8736e-01, 3.1264e-01],
        [2.2406e-16, 7.7740e-01, 2.2260e-01],
        [4.0879e-15, 9.6767e-01, 3.2331e-02],
        [1.0100e-32, 4.9796e-01, 5.0204e-01],
        [8.5821e-19, 8.2738e-01, 1.7262e-01],
        [1.0000e+00, 1.2101e-16, 1

In [11]:
predict = torch.max(predict,1)[1]
predict

tensor([0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1,
        0, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
        1, 2, 2, 2, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 2, 1, 1,
        2, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 2,
        1, 1, 1, 1])

In [12]:
df_pred = pd.DataFrame(data = {'Pred_PM2' : predict})
df_pred

,Pred_PM2
0,0
1,1
2,1
3,1
4,2
...,...
95,2
96,1
97,1
98,1


In [13]:
mergy_by_col = pd.concat([df_test, df_pred], axis=1)
mergy_by_col

,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,...,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS,Pred_PM2
0,583.44,6736.08,5728.32,1272.96,1272.96,583.44,6736.08,5728.32,1272.96,1272.96,...,437.58,5052.06,4296.24,954.72,954.72,68,23.7,0.882948,0.7,0
1,875.16,10104.12,8592.48,1909.44,1909.44,1151.92,13299.44,11309.76,2513.28,2513.28,...,583.44,6736.08,5728.32,1272.96,1272.96,34,3.5,0.190809,3.5,1
2,729.30,8420.10,7160.40,1591.20,1591.20,1092.08,12608.56,10722.24,2382.72,2382.72,...,437.58,5052.06,4296.24,954.72,954.72,58,11.9,0.754710,0.6,1
3,875.16,10104.12,8592.48,1909.44,1909.44,1600.72,18481.04,15716.16,3492.48,3492.48,...,291.72,3368.04,2864.16,636.48,636.48,25,7.0,0.275637,1.9,1
4,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,...,1301.52,15026.64,12778.56,2839.68,2839.68,66,13.7,-0.173648,0.4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,...,1155.66,13342.62,11346.48,2521.44,2521.44,68,13.1,-0.997564,0.3,2
96,875.16,10104.12,8592.48,1909.44,1909.44,583.44,6736.08,5728.32,1272.96,1272.96,...,437.58,5052.06,4296.24,954.72,954.72,58,5.9,0.544639,2.0,1
97,1021.02,11788.14,10024.56,2227.68,2227.68,437.58,5052.06,4296.24,954.72,954.72,...,0.00,0.00,0.00,0.00,0.00,48,8.4,0.515038,1.3,1
98,583.44,6736.08,5728.32,1272.96,1272.96,729.30,8420.10,7160.40,1591.20,1591.20,...,654.50,7556.50,6426.00,1428.00,1428.00,51,2.8,0.374607,0.6,1


In [ ]:
!pip install openpyxl
mergy_by_col.to_excel(excel_writer='10th_homework.xlsx')